# Navier-Stokes Type II Blowup Analysis Toolkit

This notebook demonstrates the computational toolkit for studying the Type II blowup window [3/5, 3/4).

## Contents
1. Spectral NS Solver Demo
2. Blowup Detection and Rate Fitting
3. Symbolic Identity Search
4. Visualization Tools

In [ ]:
import sys
sys.path.insert(0, '../src')

import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import ifftn

# Our modules
from simulator.spectral_ns import SpectralNSSolver, SolverConfig
from simulator.initial_conditions import taylor_green, kida_vortex, anti_parallel_vortex_tubes
from analysis.blowup_detector import BlowupDetector
from analysis.rate_tracker import RateTracker
from visualization.rate_plots import TypeIIVisualizer
from symbolic.identity_search import IdentitySearch

%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)

## 1. Spectral Solver Demo

Run a simulation with Taylor-Green initial conditions.

In [ ]:
# Configure solver
config = SolverConfig(
    N=64,           # 64³ grid
    L=2*np.pi,      # Domain [0, 2π]³
    nu=0.01,        # Viscosity
    dealias=True,   # 2/3 dealiasing
    cfl=0.5,        # CFL number
    integrator='rk4'
)

solver = SpectralNSSolver(config)
print(f"Solver initialized: {config.N}³ grid, ν = {config.nu}")

In [ ]:
# Create initial condition: Taylor-Green vortex
omega_hat_0 = taylor_green(N=config.N, L=config.L, amplitude=1.0)

# Get initial velocity for visualization
u0, v0, w0 = solver.get_velocity(omega_hat_0)

# Plot initial velocity magnitude
u_mag = np.sqrt(u0**2 + v0**2 + w0**2)
plt.figure(figsize=(8, 6))
plt.imshow(u_mag[:, :, config.N//2], cmap='viridis')
plt.colorbar(label='|u|')
plt.title('Initial velocity magnitude (z = π slice)')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

print(f"Initial ||u||_∞ = {np.max(u_mag):.4f}")

In [ ]:
# Run simulation
T_final = 5.0
print(f"Running simulation to T = {T_final}...")

history = solver.run(omega_hat_0, T_final=T_final)

print(f"Simulation complete: {len(history['times'])} steps")
print(f"Final ||u||_∞ = {history['u_Linf'][-1]:.4f}")
print(f"Final energy = {history['energy'][-1]:.6f}")

## 2. Blowup Detection and Rate Fitting

In [ ]:
# Visualize diagnostics
viz = TypeIIVisualizer()

fig = viz.plot_diagnostics(history)
plt.show()

In [ ]:
# Fit blowup rate (for demonstration - Taylor-Green decays, doesn't blow up)
tracker = RateTracker()

times = np.array(history['times'])
u_Linf = np.array(history['u_Linf'])

# Try fitting (will give non-blowup result for decaying solution)
alpha, T_star = tracker.fit_blowup_rate(times, u_Linf)
print(f"Fitted rate: α = {alpha:.4f}")
print(f"Estimated T* = {T_star:.4f}")
print(f"Classification: {tracker.classify_blowup(alpha)}")

In [ ]:
# Phase portrait
omega_Linf = np.array(history['omega_Linf'])
fig = viz.plot_phase_portrait(u_Linf, omega_Linf, times)
plt.show()

## 3. Symbolic Identity Search

Use SymPy to verify known identities and search for new monotone quantities.

In [ ]:
# Search for identities
searcher = IdentitySearch()
results = searcher.search_all()

print(searcher.summary())

In [ ]:
# Look at specific identity
energy_id = searcher.verify_known_identity('energy')
print(energy_id.notes)

In [ ]:
# Enstrophy identity (shows stretching term)
enstrophy_id = searcher.verify_known_identity('enstrophy')
print(enstrophy_id.notes)

## 4. Energy Conservation Check

Verify that our solver conserves energy correctly (up to viscous dissipation).

In [ ]:
# Energy should decay: dE/dt = -ν||∇u||²
times = np.array(history['times'])
energy = np.array(history['energy'])
dissipation = np.array(history['dissipation'])

# Numerical derivative of energy
dE_dt_numerical = np.gradient(energy, times)

# Theoretical: dE/dt = -dissipation
dE_dt_theory = -dissipation

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(times, energy, 'b-', label='Energy')
plt.xlabel('Time')
plt.ylabel('Energy')
plt.title('Energy Decay')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(times[1:], dE_dt_numerical[1:], 'b-', label='Numerical dE/dt', alpha=0.7)
plt.plot(times[1:], dE_dt_theory[1:], 'r--', label='-ν||∇u||²', alpha=0.7)
plt.xlabel('Time')
plt.ylabel('dE/dt')
plt.title('Energy Identity Check')
plt.legend()

plt.tight_layout()
plt.show()

# Check relative error
rel_error = np.abs(dE_dt_numerical[10:] - dE_dt_theory[10:]) / (np.abs(dE_dt_theory[10:]) + 1e-10)
print(f"Mean relative error in energy identity: {np.mean(rel_error):.2e}")

## 5. Search for Blowup with Different Initial Conditions

Try the anti-parallel vortex tube configuration (Hou-Luo type).

In [ ]:
# Anti-parallel vortex tubes - potential blowup candidate
config_fine = SolverConfig(
    N=64,
    L=2*np.pi,
    nu=0.001,  # Lower viscosity for more interesting dynamics
    dealias=True,
    cfl=0.3,
    integrator='rk4'
)

solver_fine = SpectralNSSolver(config_fine)

# Anti-parallel vortex tubes
omega_hat_ap = anti_parallel_vortex_tubes(
    N=config_fine.N,
    L=config_fine.L,
    amplitude=2.0,
    separation=0.25
)

print("Running anti-parallel vortex tube simulation...")
history_ap = solver_fine.run(omega_hat_ap, T_final=3.0)

print(f"Final ||u||_∞ = {history_ap['u_Linf'][-1]:.4f}")
print(f"Final ||ω||_∞ = {history_ap['omega_Linf'][-1]:.4f}")

In [ ]:
# Plot diagnostics for anti-parallel case
fig = viz.plot_diagnostics(history_ap)
plt.suptitle('Anti-Parallel Vortex Tubes (ν = 0.001)', y=1.02)
plt.show()

In [ ]:
# Fit rate and check if in Type II window
times_ap = np.array(history_ap['times'])
u_Linf_ap = np.array(history_ap['u_Linf'])

alpha_ap, T_star_ap = tracker.fit_blowup_rate(times_ap, u_Linf_ap)
print(f"\nFitted rate: α = {alpha_ap:.4f}")
print(f"Classification: {tracker.classify_blowup(alpha_ap)}")
print(f"In Type II window [3/5, 3/4): {tracker.is_in_type_ii_window(alpha_ap)}")

## Summary

This toolkit provides:

1. **Spectral NS Solver**: Pseudo-spectral solver with RK4 time integration
2. **Initial Conditions**: Taylor-Green, Kida vortex, anti-parallel tubes
3. **Blowup Detection**: Track ||u||_∞, ||ω||_∞, fit blowup rate α
4. **Symbolic Search**: Verify energy/enstrophy/helicity identities
5. **Visualization**: Rate plots, phase portraits, diagnostics

### Key Result: Type II Window [3/5, 3/4)

Any blowup (if it exists) must have rate α in this window because:
- α < 3/5 violates BKM criterion
- α ≥ 3/4 violates dissipation bounds

Use this toolkit to:
1. Search numerically for solutions approaching this window
2. Verify bounds rigorously with interval arithmetic
3. Discover new monotone quantities that might close the window